# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.020 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 31.672 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 31.137 MiB RAM in 0.126s, peaked 0.000 MiB above current, total RAM usage 62.809 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

100 loops, best of 3: 6.91 ms per loop
In [3] used 0.039 MiB RAM in 2.923s, peaked 0.000 MiB above current, total RAM usage 62.848 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [4]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 144.8)
In [4] used 0.012 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 62.859 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## Containers using the array module in Python

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
%load_ext memory_profiler

In [5] used 0.004 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 62.863 MiB


In [6]:
%memit arr = array.array('l', a)

peak memory: 78.20 MiB, increment: 15.32 MiB
In [6] used 15.488 MiB RAM in 0.417s, peaked 0.000 MiB above current, total RAM usage 78.352 MiB


7.7 ~ 15 MB vs 31 MB seems like a good deal, although sometimes Python is allocating more memory than necessary.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [7]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

16.24064

In [7] used 0.031 MiB RAM in 0.026s, peaked 0.000 MiB above current, total RAM usage 78.383 MiB


But how it performs during reductions?

In [8]:
t = %timeit -o sum(arr)

100 loops, best of 3: 11.5 ms per loop
In [8] used 0.016 MiB RAM in 4.819s, peaked 0.000 MiB above current, total RAM usage 78.398 MiB


In [9]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 86.7)
In [9] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 78.398 MiB


Well, that's a bit disappointing, as the array object performs about 2x slower than a regular array.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

## NumPy

In [10]:
import numpy as np

In [10] used 6.680 MiB RAM in 0.491s, peaked 0.000 MiB above current, total RAM usage 85.078 MiB


In [11]:
na = np.array(a, dtype=np.int64)

In [11] used 7.645 MiB RAM in 0.109s, peaked 0.000 MiB above current, total RAM usage 92.723 MiB


We see that, with 8 bytes/element, NumPy is also an efficient container (much more than the `array` package in standard library which takes ~16 bytes/element).

In [12]:
t = %timeit -o sum(na)

10 loops, best of 3: 87.9 ms per loop
In [12] used 0.016 MiB RAM in 3.713s, peaked 0.000 MiB above current, total RAM usage 92.738 MiB


In [13]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 11.381)
In [13] used 0.004 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 92.742 MiB


Oops, this is more than several times slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [14]:
t = %timeit -o na.sum()

The slowest run took 4.67 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 551 µs per loop
In [14] used 0.125 MiB RAM in 2.438s, peaked 0.000 MiB above current, total RAM usage 92.867 MiB


In [15]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 1814.092)
In [15] used 0.004 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 92.871 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [16]:
import bcolz

In [16] used 22.215 MiB RAM in 1.208s, peaked 0.000 MiB above current, total RAM usage 115.086 MiB


In [17]:
ca = bcolz.carray(na)

In [17] used 0.652 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 115.738 MiB


In [18]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.65234375)
In [18] used 0.016 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 115.754 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [19]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [19] used 0.078 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 115.832 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [20]:
t = %timeit -o ca.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 1.86 ms per loop
('MIPS:', 537.429)
In [20] used 0.059 MiB RAM in 1.041s, peaked 0.000 MiB above current, total RAM usage 115.891 MiB


This is around 2~4x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 20x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [21]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [21] used 14.785 MiB RAM in 0.104s, peaked 0.000 MiB above current, total RAM usage 130.676 MiB


In [22]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [22] used 0.008 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 130.684 MiB


In [23]:
t = %timeit -o cau.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 1.9 ms per loop
('MIPS:', 527.535)
In [23] used 0.008 MiB RAM in 0.890s, peaked 0.000 MiB above current, total RAM usage 130.691 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the only source of the overhead (and it is very minor in fact).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be competitive with NumPy, performance wise, in some scenarios.

### Exercise: Using bcolz in real scenarios

bcolz does not get good compression ratios only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics:

* Which are the typical compression ratios for this case?

* Is there a difference in speed accessing data in compressed and non-compressed state (clevel=0)

* Which are the compressors achieving the best compression/speed ratio?